In [6]:
!pip install -q selenium
!apt-get -qq update
!apt-get install -qq chromium-chromedriver sqlite3
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
urls = {
  "primary": "https://overframe.gg/tier-list/primary-weapons/",
  "secondary": "https://overframe.gg/tier-list/secondary-weapons/",
  "melee": "https://overframe.gg/tier-list/melee-weapons/"
} 

tiers = {
    "S": 1,
    "A": 2,
    "B": 3,
    "C": 4,
    "D": 5,
    "N": 6
}

In [44]:
from selenium.webdriver.common.by import By
import pandas as pd

def getTier(url, wType):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get(url)

  tierContainers = wd.find_elements(by=By.CLASS_NAME, value="TierList_tierContainer__psIt4")
  wStorage = []
  tStorage = []

  for tierContainer in tierContainers:
    currTier = tierContainer.find_element(By.CLASS_NAME, "TierList_tierTitle__PdTC9").text
    weapons = tierContainer.find_elements(By.TAG_NAME, "figcaption")
    id = tiers[str(currTier[:1])]
    for weapon in weapons:
      wStorage.append(weapon.text)
      tStorage.append(id)

  d = {
      "type": wType,
      "weapon": wStorage,
      "tier": tStorage,
  }
  df = pd.DataFrame(data=d)

  return df

### Storing data

In [54]:
ty = {
    "type": ["primary", "secondary", "melee"]
}
ti = {
    "name": ['S', 'A', 'B', 'C', 'D', 'Unknown'],
    "desc": ["Prime time", "Strong Picks", "Good Picks", "Average Picks", "Weak Picks", "New unranked items"],
    "advice": [
        "The best of the best.", 
        "Very strong, good in most situations.", 
        "Solid options for specific tasks.", 
        "These are average, or only situationally useful.", 
        "On the weak side, possible niche applications.", 
        "Vote on overframe.gg to help categorize these items!"
      ]
}
types = pd.DataFrame(data=ty)
tiers = pd.DataFrame(data=ti)

In [45]:
priFrame = getTier(urls["primary"], 1)
priFrame

,type,weapon,tier
0,1,KUVA ZARR,1
1,1,NATARUK,1
2,1,PROBOSCIS CERNOS,1
3,1,KUVA BRAMMA,1
4,1,CEDO,1
...,...,...,...
153,1,LATRON,5
154,1,BRATON,5
155,1,HIND,5
156,1,DERA,5


In [47]:
secFrame = getTier(urls["secondary"], 2)
secFrame

,type,weapon,tier
0,2,HYSTRIX PRIME,6
1,2,KUVA NUKOR,1
2,2,LAETUM,1
3,2,SPORELACER,1
4,2,TENET CYCRON,1
...,...,...,...
126,2,BALLISTICA,5
127,2,BRONCO,5
128,2,TWIN VIPERS,5
129,2,MK1-FURIS,5


In [49]:
melFrame = getTier(urls["melee"], 3)
melFrame

,type,weapon,tier
0,3,NIKANA PRIME,1
1,3,KRONEN PRIME,1
2,3,GUANDAO PRIME,1
3,3,STROPHA,1
4,3,GLAIVE PRIME,1
...,...,...,...
189,3,MK1-FURAX,5
190,3,KESTREL,5
191,3,FURAX,5
192,3,KOGAKE,5


In [50]:
frame = priFrame.append(secFrame, ignore_index=True)
frame = frame.append(melFrame, ignore_index=True)
frame.index += 1

In [51]:
frame

,type,weapon,tier
1,1,KUVA ZARR,1
2,1,NATARUK,1
3,1,PROBOSCIS CERNOS,1
4,1,KUVA BRAMMA,1
5,1,CEDO,1
...,...,...,...
479,3,MK1-FURAX,5
480,3,KESTREL,5
481,3,FURAX,5
482,3,KOGAKE,5


In [55]:
tiers.index += 1
tiers

,name,desc,advice
1,S,Prime time,The best of the best.
2,A,Strong Picks,"Very strong, good in most situations."
3,B,Good Picks,Solid options for specific tasks.
4,C,Average Picks,"These are average, or only situationally useful."
5,D,Weak Picks,"On the weak side, possible niche applications."
6,Unknown,New unranked items,Vote on overframe.gg to help categorize these ...


In [56]:
types.index += 1
types

,type
1,primary
2,secondary
3,melee


### SQL Part

In [57]:
import sqlite3
 
conn = sqlite3.connect('test.db')

In [58]:
frame.to_sql('weapons', con=conn)
types.to_sql('weapon_types', con=conn)
tiers.to_sql('weapon_tiers', con=conn)

In [59]:
!sqlite3 test.db .dump >> database.sql
!tail -n 10 database.sql

INSERT INTO weapon_types VALUES(1,'S','Prime time','The best of the best.');
INSERT INTO weapon_types VALUES(2,'A','Strong Picks','Very strong, good in most situations.');
INSERT INTO weapon_types VALUES(3,'B','Good Picks','Solid options for specific tasks.');
INSERT INTO weapon_types VALUES(4,'C','Average Picks','These are average, or only situationally useful.');
INSERT INTO weapon_types VALUES(5,'D','Weak Picks','On the weak side, possible niche applications.');
INSERT INTO weapon_types VALUES(6,'Unknown','New unranked items','Vote on overframe.gg to help categorize these items!');
CREATE INDEX "ix_weapons_index"ON "weapons" ("index");
CREATE INDEX "ix_weapon_tiers_index"ON "weapon_tiers" ("index");
CREATE INDEX "ix_weapon_types_index"ON "weapon_types" ("index");
COMMIT;
